In [75]:
import pandas as pd
import numpy as np
import json
import os

'''
playlist_col = ['collaborative', 'duration_ms', 'modified_at', 
                'name', 'num_albums', 'num_artists', 'num_edits',
                'num_followers', 'num_tracks', 'pid']
    tracks_col = ['album_name', 'album_uri', 'artist_name', 'artist_uri', 
                  'duration_ms', 'track_name', 'track_uri'] 
    playlist_test_col = ['name', 'num_holdouts', 'num_samples', 'num_tracks', 'pid']
'''

DATAFRAME_PATH = 'dataframes'

def load_playlists(path):
    for file in os.listdir("raw_data"):
        if not file.endswith(".json"):
            continue
        with open("raw_data/"+file) as f:
            js_slice = json.load(f)
            for playlist in js_slice["playlists"]:
                playlist['tracks'] = [track['track_uri'] for track in playlist['tracks']]
                yield playlist
        print(f"Added playlists from {file}")

df = pd.DataFrame(load_playlists(DATAFRAME_PATH + '/playlists.h5'))
df.to_hdf(DATAFRAME_PATH + '/playlists.h5', 'playlists')
print('Stored files to playlists.h5')

Added playlists from mpd.slice.0-999.json
Added playlists from mpd.slice.1000-1999.json
Added playlists from mpd.slice.10000-10999.json
Added playlists from mpd.slice.100000-100999.json
Added playlists from mpd.slice.101000-101999.json
Added playlists from mpd.slice.102000-102999.json
Added playlists from mpd.slice.103000-103999.json
Added playlists from mpd.slice.104000-104999.json
Added playlists from mpd.slice.105000-105999.json
Added playlists from mpd.slice.106000-106999.json
Added playlists from mpd.slice.107000-107999.json
Added playlists from mpd.slice.108000-108999.json
Added playlists from mpd.slice.109000-109999.json
Added playlists from mpd.slice.11000-11999.json
Added playlists from mpd.slice.110000-110999.json
Added playlists from mpd.slice.111000-111999.json
Added playlists from mpd.slice.112000-112999.json
Added playlists from mpd.slice.113000-113999.json
Added playlists from mpd.slice.114000-114999.json
Added playlists from mpd.slice.115000-115999.json
Added playlists 

Added playlists from mpd.slice.247000-247999.json
Added playlists from mpd.slice.248000-248999.json
Added playlists from mpd.slice.249000-249999.json
Added playlists from mpd.slice.25000-25999.json
Added playlists from mpd.slice.26000-26999.json
Added playlists from mpd.slice.27000-27999.json
Added playlists from mpd.slice.28000-28999.json
Added playlists from mpd.slice.29000-29999.json
Added playlists from mpd.slice.3000-3999.json
Added playlists from mpd.slice.30000-30999.json
Added playlists from mpd.slice.31000-31999.json
Added playlists from mpd.slice.32000-32999.json
Added playlists from mpd.slice.33000-33999.json
Added playlists from mpd.slice.34000-34999.json
Added playlists from mpd.slice.35000-35999.json
Added playlists from mpd.slice.36000-36999.json
Added playlists from mpd.slice.37000-37999.json
Added playlists from mpd.slice.38000-38999.json
Added playlists from mpd.slice.39000-39999.json
Added playlists from mpd.slice.4000-4999.json
Added playlists from mpd.slice.40000-4

E:\anaconda3\lib\site-packages\pandas\core\generic.py:2490: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['name', 'collaborative', 'tracks', 'description'], dtype='object')]

  pytables.to_hdf(


Stored files to playlists.h5


In [76]:
playlists_df = pd.read_hdf(DATAFRAME_PATH + '/playlists.h5', 'playlists')
tracks_df = playlists_df.tracks.explode().drop_duplicates()
tracks_df = tracks_df.reset_index().drop(['index'], axis=1)
tracks_df.to_hdf(DATAFRAME_PATH + '/poopoo.h5', 'tracks')

In [77]:
pd.read_hdf(DATAFRAME_PATH + "/poopoo.h5", "tracks")

,tracks
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,spotify:track:1lzr43nnXAijIGYnCT8M8H
...,...
1119305,spotify:track:6U6KdAJkDnqCc7wudXt5fY
1119306,spotify:track:5IsA9nBVVzA3ejiWDPNKxI
1119307,spotify:track:3VU8wwwmQG8lC9qoSS0AlO
1119308,spotify:track:3xLmarzSroQuXbTK44UXhD
